<center>
<h1>Хакатон Просепт х Практикум ноябрь-декабрь’23.
<br>Команда №15. Хайзенберг team. </h1>    
</center>   

**Состав команды направления DS:**      
**- Грибанов Михаил - Leader** [Telegram](https://t.me/gribanov_m) [Mail](mailto:mikhail.v.gribanov@gmail.com) Выдвижение теорий, проверка моделей.     
**- Сергеев Дмитрий**   [Telegram](https://t.me/SDI84) [Mail](mailto:dmsergeev84@mail.ru) Анализ данных, проверка моделей.  
**- Шубина Людмила** [Telegram](https://t.me/luybashu) [Mail](mailto:shubina.lu@gmail.com) Анализ данных, проверка моделей.

<a name="Содержание"></a>
#### Содержание:
- [1. Цель проекта.](#Цель_проекта)  
- [2. Первичный анализ и обработка данных.](#Первичный_анализ_данных)
 - [2.1. Первичный анализ данных.](#Первичный)
   - [2.1.1. Выводы по первичному анализу данных.](#Выводы_Первичный)
 - [2.2. Обработка данных.](#Обработка_данных)
 - [2.3. Создание тренировочной,валидационной и тестовой выборки.](#Выборки)   
- [3. Выбор и исследование моделей.](#Исследование_моделей)
 - [3.1. Метрики.](#Метрики)
 - [3.2. Модели обученные на списке товаров производимых заказчиком.](#Модели_список_заказчика)
   - [3.2.1. Модель TfidfVectorizer.](#TfidfVectorizer)
   - [3.2.2. Модель LaBSE.](#LaBSE)
   - [3.2.3. Модель LaBSE(облегченная).](#weight2)
 

<a name="Цель_проекта"></a>
## 1.Цель проекта.
Разработка решения, которое автоматизирует процесс сопоставления товаров заказчика с размещаемыми товарами дилеров.

[Полное техническое задание заказчика](https://disk.yandex.ru/i/Dt7y-TFbPEp35Q)

[Содержание](#Содержание) 

<a name="Первичный_анализ_данных"></a>
## 2.Первичный анализ и обработка данных.


Импорты необходимых билиотек:

In [1]:
# импорт библиотек
import os
import pandas as pd
import numpy as np
import torch

import re
from nltk.corpus import stopwords

# импорт моделей
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from catboost import CatBoostClassifier

# кроссвалидация
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV)

# настройки
import warnings
warnings.filterwarnings("ignore")

#import pickle

from sklearn.model_selection import train_test_split

# метрики
from sklearn.metrics import (
    roc_auc_score, 
    accuracy_score, 
    recall_score, 
    precision_score, 
    roc_curve, auc, 
    f1_score
)

In [2]:
'''
# импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# импорт моделей
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords

# кроссвалидация
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV)

# настройки
import warnings
warnings.filterwarnings("ignore")
#from skimpy import clean_columns
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

# константа верхний регистр
RANDOM_STATE = 42
'''

'\n# импорт библиотек\nimport pandas as pd\nimport numpy as np\nimport seaborn as sns\nimport matplotlib.pyplot as plt\n\n# импорт моделей\nfrom sentence_transformers import SentenceTransformer, util\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.metrics.pairwise import cosine_similarity\n\nimport nltk\nfrom nltk.corpus import stopwords\n\n# кроссвалидация\nfrom sklearn.model_selection import (\n    GridSearchCV,\n    RandomizedSearchCV)\n\n# настройки\nimport warnings\nwarnings.filterwarnings("ignore")\n#from skimpy import clean_columns\nimport logging\nlogging.getLogger(\'matplotlib.font_manager\').disabled = True\n\n# константа верхний регистр\nRANDOM_STATE = 42\n'

Задание констант.

In [2]:
#Расположение исходных файлов
DIR_DF = ['Datasets'] #путь к файлам датасетам;
FILE_DEALER = 'marketing_dealer.csv' #список дилеров;
FILE_DEALERPRICE = 'marketing_dealerprice.csv' #результат работы парсера площадок дилеров
FILE_PRODUCT = 'marketing_product.csv' #список товаров, которые производит и распространяет заказчик
FILE_PRODUCTDEALERKEY = 'marketing_productdealerkey.csv' #таблица матчинга товаров заказчика и товаров дилеров
PKL_FILE = 'pkl_model.pkl' #файл pikle с сохраненой моделью

#Зададим константу случайного распределения
RANDOM_STATE = 42

[Содержание](#Содержание) 

<a name="Первичный"></a>
### 2.1. Первичный анализ данных.

Загрузка данных в датафреймы.

In [3]:
df_dealer = pd.read_csv(os.path.join(*DIR_DF,FILE_DEALER), sep=";", decimal=",",encoding='utf-8')
df_dealerprice = pd.read_csv(os.path.join(*DIR_DF,FILE_DEALERPRICE), sep=";", decimal=",",encoding='utf-8')
df_product = pd.read_csv(os.path.join(*DIR_DF,FILE_PRODUCT), sep=";", decimal=",",encoding='utf-8')
df_productdealerkey = pd.read_csv(os.path.join(*DIR_DF,FILE_PRODUCTDEALERKEY), sep=";", decimal=",",encoding='utf-8')

Напишем функцию представления общей информации о датафрейме.

In [5]:
def express_info(data, name):
    # вывод первых строк данных
    print(f'Первые строки данных {name}:')
    display(data.head())

    # общая информация о данных
    print(f'\nОбщая информация о данных {name}:')
    data.info()

    # статистика по числовым признакам
    print(f'\nСтатистика по числовым признакам {name}:')
    display(data.describe())

    # проверка наличия пропущенных значений
    print(f'\nПропущенные значения {name}:')
    display(data.isnull().sum())

    # проверка наличия дубликатов
    duplicate_count = data.duplicated().sum()
    if duplicate_count > 0:
        print(f'\nКоличество дубликатов в данных {name}: {duplicate_count}')
    else:
        print(f'\nДубликатов в данных {name} не обнаружено.')

#### Информация о датафрейме df_product - список товаров, которые производит и распространяет заказчик.

In [6]:
express_info(df_product, 'Product')

Первые строки данных Product:


,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,0,245,008-1,4680008140234.0,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.0,858.0,20.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,189522705.0,150033482.0,008-1,NaN
1,1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN,NaN,NaN,NaN,NaN
2,2,443,0024-06 с,4680008145208.0,"Герметик акриловый цвет сосна, ф/п 600мл",307.0,644.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...,189522735.0,150126217.0,0024-06-с,NaN
3,3,147,305-2,4610093420164.0,Кондиционер для белья с ароматом королевского...,157.73,342.0,29.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...",339377922.0,150032962.0,305-2,NaN
4,4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б,NaN



Общая информация о данных Product:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         496 non-null    int64 
 1   id                 496 non-null    int64 
 2   article            496 non-null    object
 3   ean_13             464 non-null    object
 4   name               494 non-null    object
 5   cost               491 non-null    object
 6   recommended_price  491 non-null    object
 7   category_id        447 non-null    object
 8   ozon_name          458 non-null    object
 9   name_1c            485 non-null    object
 10  wb_name            455 non-null    object
 11  ozon_article       365 non-null    object
 12  wb_article         340 non-null    object
 13  ym_article         337 non-null    object
 14  wb_article_td      32 non-null     object
dtypes: int64(2), object(13)
memory usage: 58.2+ KB

Статист

,Unnamed: 0,id
count,496.000000,496.000000
mean,247.500000,251.320565
std,143.327132,147.092825
min,0.000000,1.000000
25%,123.750000,124.750000
50%,247.500000,248.500000
75%,371.250000,377.250000
max,495.000000,513.000000



Пропущенные значения Product:


Unnamed: 0             0
id                     0
article                0
ean_13                32
name                   2
cost                   5
recommended_price      5
category_id           49
ozon_name             38
name_1c               11
wb_name               41
ozon_article         131
wb_article           156
ym_article           159
wb_article_td        464
dtype: int64


Дубликатов в данных Product не обнаружено.


Отдельно выведем значения столбцов 'name'. 

In [7]:
for i in ['name','ozon_name','name_1c','wb_name']:
    print(i,'\n',df_product[i].iloc[:5])

name 
 0    Антисептик невымываемыйPROSEPT ULTRAконцентрат...
1       Антигололед - 32 PROSEPTготовый состав / 12 кг
2             Герметик акриловый цвет сосна, ф/п 600мл
3    Кондиционер для белья с ароматом  королевского...
4                 Герметик акриловой  цвет Белый, 7 кг
Name: name, dtype: object
ozon_name 
 0    Антисептик невымываемый для ответственных конс...
1                                                  NaN
2    Герметик акриловый для швов для деревянных дом...
3    Кондиционер для белья "Королевский Ирис" Prose...
4                                                  NaN
Name: ozon_name, dtype: object
name_1c 
 0    Антисептик невымываемый для ответственных конс...
1       Антигололед - 32 PROSEPTготовый состав / 12 кг
2             Герметик акриловый цвет сосна, ф/п 600мл
3    Кондиционер для белья "Королевский Ирис" Prose...
4                                                  NaN
Name: name_1c, dtype: object
wb_name 
 0    Антисептик невымываемый для ответственных кон

Поищем дубликаты по столбцам 'name' и 'article'.

In [8]:
print(f"Уникальные значения в столбце 'id': {df_product['id'].nunique()}")
print(f"Уникальные значения в столбце 'article': {df_product['article'].nunique()}")
print(f"Уникальные значения в столбце 'name': {df_product['name'].nunique()}")
print(f"Уникальные значения в столбце 'ean_13': {df_product['ean_13'].nunique()}")

Уникальные значения в столбце 'id': 496
Уникальные значения в столбце 'article': 496
Уникальные значения в столбце 'name': 487
Уникальные значения в столбце 'ean_13': 464


Выведем дубликаты по столбцу 'name'

In [9]:
df_product[df_product.duplicated('name', keep=False)].sort_values(by='name')

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
4,4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б,NaN
469,469,363,0024-7б,4680008148872.0,"Герметик акриловой цвет Белый, 7 кг",3149.0,5492.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловой цвет Белый, 7 кг",Герметик акриловый для швов для деревянных дом...,NaN,NaN,NaN,NaN
78,78,362,0024-3б,NaN,"Герметик акриловый цвет Белый, 3 кг",1251.0,2145.0,25.0,NaN,"Герметик акриловый цвет Белый, 3 кг",NaN,NaN,NaN,NaN,NaN
234,234,223,0024-3 б,4610093420485.0,"Герметик акриловый цвет Белый, 3 кг",1251.0,2145.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет Белый, 3 кг",Герметик акриловый для швов для деревянных дом...,453200111.0,150126218.0,0024-3-б,NaN
94,94,366,0024-3м,NaN,"Герметик акриловый цвет Медовый, 3 кг",1251.0,2145.0,25.0,NaN,"Герметик акриловый цвет Медовый, 3 кг",NaN,NaN,NaN,NaN,NaN
230,230,225,0024-3 м,4610093420508.0,"Герметик акриловый цвет Медовый, 3 кг",1251.0,2145.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет Медовый, 3 кг",Герметик акриловый для швов для деревянных дом...,453214675.0,150126205.0,0024-3-м,NaN
83,83,374,0024-3о,NaN,"Герметик акриловый цвет Орех, 3 кг",1251.0,2145.0,25.0,NaN,"Герметик акриловый цвет Орех, 3 кг",NaN,NaN,NaN,NaN,NaN
224,224,224,0024-3 о,4610093420492.0,"Герметик акриловый цвет Орех, 3 кг",1251.0,2145.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет Орех, 3 кг",Герметик акриловый для швов для деревянных дом...,453205718.0,150126215.0,0024-3-о,NaN
84,84,370,0024-3с,NaN,"Герметик акриловый цвет Сосна, 3 кг",1251.0,2145.0,25.0,NaN,"Герметик акриловый цвет Сосна, 3 кг",NaN,NaN,NaN,NaN,NaN
225,225,226,0024-3 с,4610093420515.0,"Герметик акриловый цвет Сосна, 3 кг",1251.0,2145.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет Сосна, 3 кг",Герметик акриловый для швов для деревянных дом...,453215204.0,150126208.0,0024-3-с,NaN


Выводы по датафрейму 'df_product'.  
 - Столбцы которые можно использовать как признаки при обучении моделей name, cost, min_recommended_price,recommended_price,     category_id, ozon_name, name_1c, wb_name, содержат пропуски.  
 - Информация представленная в столбцах name не отформатировнна встречаются объединненые слова.  
 - В информации в столбцах article встречатся разный формат записи одного артикля(добавляется пробел).  
 - В столбцах name также присутствуют данные о объеме(весе) продукта, что может быть использовано в качестве признака для модели.  
 - При этом все дублированные записи имеют разные id-ключ который используется в таблице матчинга товаров заказчика и дилера.  

#### Информация о датафрейме df_dealer - список дилеров.

In [10]:
express_info(df_dealer, 'Dealer')

Первые строки данных Dealer:


,id,name
0,1,Moi_vibor_WB
1,2,Akson
2,3,Bafus
3,5,Castorama
4,6,Cubatora



Общая информация о данных Dealer:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18 non-null     int64 
 1   name    18 non-null     object
dtypes: int64(1), object(1)
memory usage: 416.0+ bytes

Статистика по числовым признакам Dealer:


,id
count,18.000000
mean,9.500000
std,5.338539
min,1.000000
25%,5.250000
50%,9.500000
75%,13.750000
max,18.000000



Пропущенные значения Dealer:


id      0
name    0
dtype: int64


Дубликатов в данных Dealer не обнаружено.


In [11]:
df_dealer

,id,name
0,1,Moi_vibor_WB
1,2,Akson
2,3,Bafus
3,5,Castorama
4,6,Cubatora
5,7,Komus
6,9,Megastroy
7,10,OnlineTrade
8,11,Petrovich
9,12,sdvor


Выводы по датафрейму 'df_dealer'.  
 - аномалий не обнаружено. Только запись spb_Ozon содержит непечатыемые символы, возможно произошла ошибка при выгрузке.

#### Информация о датафрейме df_dealerprice - результат работы парсера площадок дилеров.

In [12]:
express_info(df_dealerprice, 'Dealerprice')

Первые строки данных Dealerprice:


,id,product_key,price,product_url,product_name,date,dealer_id
0,2,546227,233.00,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2
1,3,546408,175.00,https://akson.ru//p/kontsentrat_prosept_multip...,"Концентрат Prosept Multipower для мытья полов,...",2023-07-11,2
2,4,546234,285.00,https://akson.ru//p/sredstvo_dlya_chistki_lyus...,Средство для чистки люстр Prosept Universal An...,2023-07-11,2
3,5,651258,362.00,https://akson.ru//p/udalitel_rzhavchiny_prosep...,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0...",2023-07-11,2
4,6,546355,205.00,https://akson.ru//p/sredstvo_moyushchee_dlya_b...,Средство моющее для бани и сауны Prosept Multi...,2023-07-11,2



Общая информация о данных Dealerprice:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20416 entries, 0 to 20415
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            20416 non-null  int64 
 1   product_key   20416 non-null  object
 2   price         20416 non-null  object
 3   product_url   20182 non-null  object
 4   product_name  20416 non-null  object
 5   date          20416 non-null  object
 6   dealer_id     20416 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 1.1+ MB

Статистика по числовым признакам Dealerprice:


,id,dealer_id
count,20416.000000,20416.000000
mean,10350.362363,8.938627
std,5912.127932,5.723330
min,2.000000,1.000000
25%,5257.750000,3.000000
50%,10361.500000,9.000000
75%,15465.250000,15.000000
max,20570.000000,18.000000



Пропущенные значения Dealerprice:


id                0
product_key       0
price             0
product_url     234
product_name      0
date              0
dealer_id         0
dtype: int64


Дубликатов в данных Dealerprice не обнаружено.


In [13]:
print(f"Уникальные значения в столбце 'product_key': {df_dealerprice['product_key'].nunique()}")
print(f"Уникальные значения в столбце 'product_name': {df_dealerprice['product_name'].nunique()}")
print(f"Уникальные значения в столбце 'id': {df_dealerprice['id'].nunique()}")

Уникальные значения в столбце 'product_key': 1965
Уникальные значения в столбце 'product_name': 1953
Уникальные значения в столбце 'id': 20416


In [14]:
non_unique_rows = df_dealerprice[~df_dealerprice.duplicated(subset=['product_key','product_name'], keep=False)].sort_values(by='product_key')
non_unique_rows

,id,product_key,price,product_url,product_name,date,dealer_id
20132,20284,382745,1771.00,https://akson.ru//p/ognebiozaschita_prosept_pr...,Огнебиозащита PROSEPT prof 1 группа для наружн...,2023-07-31,2
15443,15492,382753,683.00,https://akson.ru//p/antiseptik_prosept_univers...,Пропитка-антисептик PROSEPT Universal универса...,2023-07-26,2
20152,20304,546235,223.00,https://akson.ru//p/sredstvo_kontsentrirovanno...,Средство концентрированное Prosept Bath Fungi ...,2023-07-31,2
20147,20299,546293,385.00,https://akson.ru//p/sredstvo_dlya_udaleniya_gr...,"Средство для удаления граффити, маркера, краск...",2023-07-31,2
20118,20270,546323,336.00,https://akson.ru//p/udalitel_tsementa_prosept_...,"Удалитель цемента Prosept Cement Cleaner, 500м...",2023-07-31,2
...,...,...,...,...,...,...,...
20284,20439,900996549,294.00,https://www.ozon.ru/product/900996549,Отбеливатель для древесины PROSEPT ECO 50 гото...,2023-07-31,18
6240,6332,928819,318.00,https://vegosm.ru/catalog/interer_i_otdelka/la...,PROSEPT FUNGI CLEAN - удалитель плесени с мине...,2023-07-14,14
19198,19348,https://kub02.ru/catalog/emali/propitka_dlya_k...,406.00,https://kub02.ru/catalog/emali/propitka_dlya_k...,Пропитка для камня гидрофобизирующий состав (1...,2023-07-31,6
19197,19347,https://kub02.ru/catalog/professionalnaya_byto...,396.00,https://kub02.ru/catalog/professionalnaya_byto...,Удалитель ржавчины PROSEPT RUST REMOVER готовы...,2023-07-31,6


Выводы по датафрейму 'df_dealerprice'.
- Столбцы которые можно использовать как признаки при обучении моделей price, priduct_name,dealer_id.
- Имеется большое количество дубликатов по столбцу product_key, product_name.

#### Информация о датафрейме df_productdealerkey - таблица матчинга товаров заказчика и товаров дилеров.

In [15]:
express_info(df_productdealerkey, 'Productdealerkey')

Первые строки данных Productdealerkey:


,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106
2,3,546257,2,200
3,4,546408,2,38
4,5,651258,2,403



Общая информация о данных Productdealerkey:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1700 non-null   int64 
 1   key         1700 non-null   object
 2   dealer_id   1700 non-null   int64 
 3   product_id  1700 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 53.2+ KB

Статистика по числовым признакам Productdealerkey:


,id,dealer_id,product_id
count,1700.000000,1700.000000,1700.000000
mean,1063.140588,10.148824,223.564118
std,615.283828,5.830717,132.341778
min,1.000000,1.000000,1.000000
25%,425.750000,3.000000,110.000000
50%,1173.500000,10.000000,239.500000
75%,1598.250000,15.000000,334.250000
max,2023.000000,18.000000,479.000000



Пропущенные значения Productdealerkey:


id            0
key           0
dealer_id     0
product_id    0
dtype: int64


Дубликатов в данных Productdealerkey не обнаружено.


In [16]:
print(f"Уникальные значения в столбце 'id': {df_productdealerkey['id'].nunique()}")
print(f"Уникальные значения в столбце 'product_id': {df_productdealerkey['product_id'].nunique()}")

Уникальные значения в столбце 'id': 1700
Уникальные значения в столбце 'product_id': 438


Рассмотрим есть ли все значения 'key' в датафрейме df_dealerprice, 'product_id' в датафрейме df_product, 'dealer_id' в датафрейме  df_dealer.

In [17]:
print(f"Число значений столбца 'key' отсутствующие в df_dealerprice:",\
      len(set(df_productdealerkey['key'].unique())-set(df_dealerprice['product_key'].unique())))
print(f"Число значений столбца 'product_id' отсутствующие в df_product:",\
     len(set(df_productdealerkey['product_id'].unique())-set(df_product['id'].unique())))
print(f"Число значений столбца 'dealer_id' отсутствующие в df_dealer",\
     len(set(df_productdealerkey['dealer_id'].unique())-set(df_dealer['id'].unique())))

Число значений столбца 'key' отсутствующие в df_dealerprice: 98
Число значений столбца 'product_id' отсутствующие в df_product: 0
Число значений столбца 'dealer_id' отсутствующие в df_dealer 0


Выводы по датафрейму 'df_productdealerkey'.
- Имеются ключи, отсутствующие во внешних датафреймах.

[Содержание](#Содержание) 

<a name="Выводы_Первичный"></a>
#### 2.1.1. Выводы по первичному анализу данных.
Выводы по датафрейму 'df_product'.  
 - Столбцы которые можно использовать как признаки при обучении моделей name, cost, min_recommended_price,recommended_price,     category_id, ozon_name, name_1c, wb_name, содержат пропуски.  
 - Информация представленная в столбцах name не отформатирована встречаются объединненые слова.  
 - В информации в столбцах article встречатся разный формат записи одного артикля(добавляется пробел).  
 - В столбцах name также присутствуют данные о объеме(весе) продукта, что может быть использовано в качестве признака для модели.  
 - При этом все дублированные записи имеют разные id-ключ который используется в таблице матчинга товаров заказчика и дилера.   


Выводы по датафрейму 'df_dealer'.    
- аномалий не обнаружено. Только запись spb_Ozon содержит непечатыемые символы, возможно произошла ошибка при выгрузке.

Выводы по датафрейму 'df_dealerprice'.
- Столбцы которые можно использовать как признаки при обучении моделей price, priduct_name,dealer_id.
- Имеется большое количество дубликатов по столбцу product_key, product_name.

Выводы по датафрейму 'df_productdealerkey'.
- Имеются ключи, отсутствующие во внешних датафреймах.

***Задачи на обработку данных:***  
- ***для df_product*** удалим из df_name строки в которых отсутствует значение в столбце 'name'. Так же требуется написать функцию по обработке тектса столбцов 'name' (разделение слов, привод к одному размеру, удаление лишних пробелов, выделение веса(объема) товаров    
- ***для df_dealer*** обработки не требуется.
- ***для df_dealerprice*** удалить дубликаты по столбцу product_key.  
- ***df_productdealerkey*** удалить строки содержащие ключи отсутствующие во внешних датафреймах. Создать на основе датафрейма -тестовый с добавлением признаков из внешних таблиц.  

[Содержание](#Содержание) 

<a name="Обработка_данных"></a>
### 2.2. Обработка данных.

Напишем функцию обработки строк 'name' в датафреймах df_dealerprice, df_product.  

Загрузим словарь со стоп словами.

In [4]:
# стоп-слова для английского и русского языков
stop_words_en = set(stopwords.words('english'))
stop_words_ru = set(stopwords.words('russian'))

# объединим стоп-слова
stop_words = stop_words_en.union(stop_words_ru)

#### первая функция clean_text()
Функция clean_texts очищает строки от маркеры переноса строки и табуляции, гиперссылки, знаки пунктуации, стоп слов, приводит строку к нижнеу регистру.

In [19]:
def clean_texts(texts):
    
    if not pd.isna(texts):
        # нижний регистр
        texts = texts.lower()

        # замена всех вхождений \n,\\n,\t,\\ на пробел
        texts = (texts.replace('\\n', ' ')
                 .replace('\n', ' ')
                 .replace('\n\n', ' ')
                 .replace('\t',' ')
                 .replace('\\', ' '))

        # удаление всех вхождений ссылок, начинающихся с "https"
        texts = texts.replace('http\S+', '')

        # удаление текста, заканчивающегося на ".com"
        texts = texts.replace('\ [a-z]*\.com', ' ')

        # удаление знаков препинания (кроме ')
        texts = texts.replace("[^а-яa-z\d\s']+", ' ')

        texts = texts.replace(r'\bprosept\b', '')

        # удаление стоп-слов
        texts = ' '.join([word for word in texts.split() if word not in stop_words])
        #texts = texts.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    else:
        texts=''
    
    return texts

Вариант функции для Series.
```python

def clean_texts(texts):
    # нижний регистр
    texts = texts.str.lower()
           
    # замена всех вхождений \n,\\n,\t,\\ на пробел
    texts = (texts.str.replace('\\n', ' ')
             .str.replace('\n', ' ')
             .str.replace('\n\n', ' ')
             .str.replace('\t',' ')
             .str.replace('\\', ' '))
    
    # удаление всех вхождений ссылок, начинающихся с "https"
    texts = texts.str.replace('http\S+', '')
    
    # удаление текста, заканчивающегося на ".com"
    texts = texts.str.replace('\ [a-z]*\.com', ' ')
    
    # удаление знаков препинания (кроме ')
    texts = texts.str.replace("[^а-яa-z\d\s']+", ' ')
    
    texts = texts.str.replace(r'\bprosept\b', '')
    
    # удаление стоп-слов
    texts = texts.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    
    return texts
```

Проверим работу функции.

In [20]:
for i in df_product['name'].loc[:5]:
    print(i)

Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10  / 1 л
Антигололед - 32 PROSEPTготовый состав / 12 кг
Герметик акриловый цвет сосна, ф/п 600мл
Кондиционер для белья с ароматом  королевского ИрисаCrystal Rinserконцентрат / 2 л 
Герметик акриловой  цвет Белый, 7 кг
Грунт БЕТОНКОНТАКТготовый состав / 6 кг


In [21]:
for i in df_product['name'].loc[:5].apply(clean_texts):
    print(i)

антисептик невымываемыйprosept ultraконцентрат 1:10 / 1 л
антигололед - 32 proseptготовый состав / 12 кг
герметик акриловый цвет сосна, ф/п 600мл
кондиционер белья ароматом королевского ирисаcrystal rinserконцентрат / 2 л
герметик акриловой цвет белый, 7 кг
грунт бетонконтактготовый состав / 6 кг


#### Вторая функция str_edit()
Функция str_edit разделяет объединенные русские и английские слова, удаляет лишние пробелы.

In [22]:
def str_edit(res):
    if not pd.isna(res):
        res = ' '.join(re.split(r"([A-Za-z][A-Za-z]*)",res))
        #res = ' '.join(re.split(r"\b[А-ЯЁ]([А-ЯЁ][А-ЯЁа-яё]*)",res))
        res = ' '.join(re.split(r"([0-9][0-9]*)",res))
        res = ' '.join(res.split()).lower()
    else:
        res = ''
        
    return res

Проверим работу функции.

In [23]:
for i in df_product['name'].loc[:5]:
    print(i)

Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10  / 1 л
Антигололед - 32 PROSEPTготовый состав / 12 кг
Герметик акриловый цвет сосна, ф/п 600мл
Кондиционер для белья с ароматом  королевского ИрисаCrystal Rinserконцентрат / 2 л 
Герметик акриловой  цвет Белый, 7 кг
Грунт БЕТОНКОНТАКТготовый состав / 6 кг


In [24]:
for i in df_product['name'].loc[:5].apply(str_edit):
    print(i)

антисептик невымываемый prosept ultra концентрат 1 : 10 / 1 л
антигололед - 32 prosept готовый состав / 12 кг
герметик акриловый цвет сосна, ф/п 600 мл
кондиционер для белья с ароматом королевского ириса crystal rinser концентрат / 2 л
герметик акриловой цвет белый, 7 кг
грунт бетонконтактготовый состав / 6 кг


#### Третья функция str_edit2()
Функция str_edit2 разделяет объединенные русские и английские слова, удаляет лишние пробелы, удаляет знаки препинания.

In [25]:
def str_edit2(res):
    if not pd.isna(res):
        res = re.sub(r'[^\w\s]','', res)
        res = ' '.join(re.split(r"([A-Za-z][A-Za-z]*)",res))
        #res = ' '.join(re.split(r"\b[А-ЯЁ]([А-ЯЁ][А-ЯЁа-яё]*)",res))
        res = ' '.join(re.split(r"([0-9][0-9]*)",res))
        res = ' '.join(res.split()).lower()
    else:
        res = ''
        
    return res

In [26]:
for i in df_product['name'].loc[:5].apply(str_edit2):
    print(i)

антисептик невымываемый prosept ultra концентрат 110 1 л
антигололед 32 prosept готовый состав 12 кг
герметик акриловый цвет сосна фп 600 мл
кондиционер для белья с ароматом королевского ириса crystal rinser концентрат 2 л
герметик акриловой цвет белый 7 кг
грунт бетонконтактготовый состав 6 кг


#### Объединенные функция str_edit2_clean() и str_edit_clean()

In [27]:
def str_edit2_clean(res):
    return clean_texts(str_edit2(res))
    
def str_edit_clean(res):
    return clean_texts(str_edit(res))

In [28]:
for i in df_product['name'].loc[:5].apply(str_edit2_clean):
    print(i)
for i in df_product['name'].loc[:5].apply(str_edit_clean):
    print(i)    

антисептик невымываемый prosept ultra концентрат 110 1 л
антигололед 32 prosept готовый состав 12 кг
герметик акриловый цвет сосна фп 600 мл
кондиционер белья ароматом королевского ириса crystal rinser концентрат 2 л
герметик акриловой цвет белый 7 кг
грунт бетонконтактготовый состав 6 кг
антисептик невымываемый prosept ultra концентрат 1 : 10 / 1 л
антигололед - 32 prosept готовый состав / 12 кг
герметик акриловый цвет сосна, ф/п 600 мл
кондиционер белья ароматом королевского ириса crystal rinser концентрат / 2 л
герметик акриловой цвет белый, 7 кг
грунт бетонконтактготовый состав / 6 кг


#### Вспомогательная функция no_change()
Функция no_change не меняет исходный текст, но заменяет NaN на ''.

In [29]:
def no_change(res):
    res=res
    if pd.isna(res):
        res = ''
    return res

Для проведения тестов зададим список из функций.

In [30]:
list_func = [str_edit_clean,str_edit2_clean,clean_texts,no_change,str_edit,str_edit2]

Удалим из датафрейма df_product строки в которых отсутствует значения 'name'.

In [5]:
df_product=df_product.dropna(subset=['name'])
df_product.reset_index(drop= True,inplace=True)
df_product.head()

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,0,245,008-1,4680008140234.0,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.0,858.0,20.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,189522705.0,150033482.0,008-1,NaN
1,1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN,NaN,NaN,NaN,NaN
2,2,443,0024-06 с,4680008145208.0,"Герметик акриловый цвет сосна, ф/п 600мл",307.0,644.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...,189522735.0,150126217.0,0024-06-с,NaN
3,3,147,305-2,4610093420164.0,Кондиционер для белья с ароматом королевского...,157.73,342.0,29.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...",339377922.0,150032962.0,305-2,NaN
4,4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б,NaN


[Содержание](#Содержание) 

<a name="Выборки"></a>
### 2.3. Создание тренировочной,валидационной и тестовой выборки.

Созданим тестовую таблицу df_all_test объединив по внешним ключам к таблице df_productdealerkey данные из таблиц df_product, df_dealerprice.

In [6]:
#Создаем тестовый датафрейм для проверки результатов
df_all_test = df_productdealerkey.copy()
df_test_dealerprice = df_dealerprice[['product_key','dealer_id','product_name']].copy()
df_test_dealerprice.columns = ['key','dealer_id','product_name']
df_all_test = df_all_test.merge(df_test_dealerprice,how='left',on=['key','dealer_id']).drop_duplicates().dropna()
df_all_test = df_all_test.merge(df_product.reset_index()[['index','id','name','ozon_name','name_1c','wb_name']],how='left',left_on=['product_id'],right_on=['id'])\
                .drop_duplicates()

In [133]:
express_info(df_all_test, 'all_test')

Первые строки данных all_test:


,id_x,key,dealer_id,product_id,product_name,index,id_y,name,ozon_name,name_1c,wb_name
0,1,546227,2,12,"Средство универсальное Prosept Universal Spray, 500мл",486,12,"Универсальное моющее и чистящее средствоUniversal Sprayготовый состав / 0,5 л","Универсальное чистящее средство PROSEPT Universal Spray, 0.5 л.","Универсальное чистящее средство PROSEPT Universal Spray, 0.5 л.","Универсальное чистящее средство PROSEPT Universal Spray, 0.5 л."
1,2,651265,2,106,"Соль для посудомоечных машин PROSEPT Splash 1,5кг 280-15",455,106,"Специальная соль для посудомоечных машин Splashготовый состав / 1,5 кг","Соль для посудомоечных машин PROSEPT Splash, 1.5 кг.","Соль для посудомоечных машин PROSEPT Splash, 1.5 кг.","Соль для посудомоечных машин PROSEPT Splash, 1.5 кг."
2,3,546257,2,200,"Средство для мытья стекол и зеркал Prosept Optic Crystal, 500мл",315,200,"Средство для мытья стекол и зеркалOptic Crystalготовый состав / 0,5 л","Средство для мытья стекол, зеркал и пластика PROSEPT Optic Crystal, 500 мл.","Средство для мытья стекол, зеркал и пластика PROSEPT Optic Crystal, 500 мл.","Средство для мытья стекол, зеркал и пластика PROSEPT Optic Crystal, 500 мл."
3,4,546408,2,38,"Концентрат Prosept Multipower для мытья полов, цитрус 1л",419,38,Концентрат для мытья половMULTIPOWER с ароматом цитрусаконцентрат / 1 л,"Средство для мытья полов PROSEPT Multipower citrus, 1л.","Средство для мытья полов PROSEPT Multipower citrus, 1л.","Средство для мытья полов PROSEPT Multipower citrus, 1л."
4,5,651258,2,403,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05",282,403,"Удалитель ржавчины RUST REMOVER готовый состав / 0,5 л","Удалитель ржавчины PROSEPT RUST REMOVER, 0.5 л.","Удалитель ржавчины PROSEPT RUST REMOVER, 0.5 л.","Удалитель ржавчины PROSEPT RUST REMOVER, 0.5 л."



Общая информация о данных all_test:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1691 entries, 0 to 1690
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_x          1691 non-null   int64 
 1   key           1691 non-null   object
 2   dealer_id     1691 non-null   int64 
 3   product_id    1691 non-null   int64 
 4   product_name  1691 non-null   object
 5   index         1691 non-null   int64 
 6   id_y          1691 non-null   int64 
 7   name          1691 non-null   object
 8   ozon_name     1637 non-null   object
 9   name_1c       1684 non-null   object
 10  wb_name       1637 non-null   object
dtypes: int64(5), object(6)
memory usage: 158.5+ KB

Статистика по числовым признакам all_test:


,id_x,dealer_id,product_id,index,id_y
count,1691.000000,1691.000000,1691.000000,1691.000000,1691.000000
mean,1074.044352,9.859846,222.628622,298.063276,222.628622
std,624.492058,5.830433,133.185267,133.232169,133.185267
min,1.000000,1.000000,1.000000,0.000000,1.000000
25%,419.500000,3.000000,106.000000,202.500000,106.000000
50%,1181.000000,10.000000,237.000000,320.000000,237.000000
75%,1653.500000,15.000000,329.000000,407.000000,329.000000
max,2023.000000,18.000000,479.000000,493.000000,479.000000



Пропущенные значения all_test:


id_x             0
key              0
dealer_id        0
product_id       0
product_name     0
index            0
id_y             0
name             0
ozon_name       54
name_1c          7
wb_name         54
dtype: int64


Дубликатов в данных all_test не обнаружено.


Разбиение выборок на тренировочную,валидационную и тестовую.

In [7]:
df_train_valid, df_test = train_test_split(df_all_test,random_state=RANDOM_STATE,test_size=0.33)
df_train,df_valid = train_test_split(df_train_valid,random_state=RANDOM_STATE,test_size=0.5)
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 566 entries, 157 to 402
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_x          566 non-null    int64 
 1   key           566 non-null    object
 2   dealer_id     566 non-null    int64 
 3   product_id    566 non-null    int64 
 4   product_name  566 non-null    object
 5   index         566 non-null    int64 
 6   id_y          566 non-null    int64 
 7   name          566 non-null    object
 8   ozon_name     550 non-null    object
 9   name_1c       565 non-null    object
 10  wb_name       550 non-null    object
dtypes: int64(5), object(6)
memory usage: 53.1+ KB


[Содержание](#Содержание) 

<a name="Исследование_моделей"></a>
## 3.Исследование моделей.

<a name="Метрики"></a>
### 3.1. Метрики.

Модель будет возвращать нам список с предложенными вариантами. В качестве метрики будем использовать:    
Accuracy_in1 - число правильных ответов из первого предложенного;   
Accuracy_in5 - число правильных ответов из пяти предложенных;  
MRR(Mean Reciprocal Rank) - метрика оценивающая расположение правильного ответа относительно первой позиции;  

Для хранения полученных метрик используем датафрейм df_result_valid.

In [8]:
df_result_valid = pd.DataFrame(columns=['Имя модели','Дообучение','Accuracy_in1','Accuracy_in5','MRR'])
df_result_valid['Дообучение'] = df_result_valid['Дообучение'].astype(bool)
df_result_valid.head()

,Имя модели,Дообучение,Accuracy_in1,Accuracy_in5,MRR


[Содержание](#Содержание) 

<a name="Модели_список_заказчика"></a>
### 3.2. Модели обученные на списке товаров производимых заказчиком.

Проверим модели:
- TfidfVectorizer;
- LaBSE;

<a name="TfidfVectorizer"></a>
#### 3.2.1. Модель TfidfVectorizer.

Модель рассматривалась в бокноте Procept_TF-IDF_LaBSE.ipynb, результаты хуже модели LaBSE.

[Содержание](#Содержание) 

<a name="LaBSE"></a>
#### 3.2.2. Модель LaBSE.

Загрузка предварительно обученной модели LaBSE

In [36]:
model_LaBSE = SentenceTransformer('LaBSE')

Проверим модель при разной обработке текста и разных столбцах для обучения.

In [37]:
%%time

list_columns = [['name','ozon_name','name_1c','wb_name'],['name']]

for func in list_func:
    for columns in list_columns:
        
        
        def t_fit_LaBSE(df,func,df_columns=['name']):
            
            df_tmp = df[df_columns[0]].apply(func)
            
            if len(df_columns)>1:
                for i in range(1,len(df_columns)):
                    df_tmp = df_tmp + ' ' + df[df_columns[i]].apply(func)

            model = model_LaBSE.encode(df_tmp)

            return model,df[['id','name']]

        #Получим вектора из датафрейма df_product.
        product_embedding_LaBSE, df_product_LaBSE =  t_fit_LaBSE(df_product,func,columns)
    
        def t_predict_LaBSE(txt):
            txt_embedding = model_LaBSE.encode(func(txt))
            return util.pytorch_cos_sim(txt_embedding,product_embedding_LaBSE)

        df_predict_LaBSE = df_valid['product_name'].apply(t_predict_LaBSE)

        # индексы лучших совпадений для строк
        top_k_matches = []
        for i in range(df_predict_LaBSE.shape[0]):
            _,indices = df_predict_LaBSE.iloc[i].topk(len(df_product))
            top_k_matches.append(indices)
        
        #Вычислим метрики для данной модели.
        df_predict = pd.DataFrame()
        df_predict['index'] = df_valid['index']
        df_predict['predict'] = top_k_matches

        n=5
        def t_compare_LaBSE(df):

            return (df['index'] in (df['predict'][0][0:n]))
        n=1
        accuracy_in1_LaBSE='{:.3f}'.format(df_predict[['index','predict']].apply(t_compare_LaBSE,axis=1).sum()*100/df_predict.shape[0])
        n=5
        accuracy_in5_LaBSE='{:.3f}'.format(df_predict[['index','predict']].apply(t_compare_LaBSE,axis=1).sum()*100/df_predict.shape[0])


        def t_compare_MRR_LaBSE(df):
            return (1/(list(df['predict'][0]).index(df['index'])+1))

        MRR_LaBSE = '{:.3f}'.format(df_predict[['index','predict']].apply(t_compare_MRR_LaBSE,axis=1).sum()/df_predict.shape[0])
        print(accuracy_in1_LaBSE,accuracy_in5_LaBSE, MRR_LaBSE)
        tmp_str = 'all' if len(columns)>1 else 'name'
        df_result_valid = pd.concat([df_result_valid, pd.DataFrame({'Имя модели':[\
                        f'Модель LaBSE c функцией очистки:{func.__name__},столбцы:{tmp_str}']\
                        ,'Дообучение':[False]\
                        ,'Accuracy_in1':[accuracy_in1_LaBSE]\
                        ,'Accuracy_in5':[accuracy_in5_LaBSE]\
                        ,'MRR':[MRR_LaBSE]})],ignore_index=True)

74.028 92.580 0.823
67.314 90.989 0.781
71.201 91.873 0.805
68.728 90.636 0.780
72.615 92.049 0.812
63.251 88.163 0.745
71.025 92.049 0.804
61.661 89.399 0.743
73.322 93.110 0.820
65.194 90.636 0.768
72.085 92.580 0.811
66.961 90.459 0.769
CPU times: total: 1h 46min 39s
Wall time: 10min 48s
Compiler : 156 ms


[Содержание](#Содержание) 

<a name="weight2"></a>
#### 3.2.3. Модель LaBSE2.

In [14]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")
sentences = ["Hello World", "Привет Мир"]
encoded_input = tokenizer(sentences
                          , padding=True
                          , truncation=True
                          , max_length=64
                          , return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [145]:
torch.__version__

'2.1.1+cpu'

In [9]:
def clean_texts(name):
    '''
    Функции очистки текста.
    Принимает на вход строку - название товара,
    возвращает его в отредактированном виде
    '''
    
    # стоп-слова для английского и русского языков
    stop_words_en = set(stopwords.words('english'))
    stop_words_ru = set(stopwords.words('russian'))
    # объединим стоп-слова
    stop_words = stop_words_en.union(stop_words_ru)
    
    if not pd.isna(name):
        # разделение слов
        name = ' '.join(re.split(r"([A-Za-z][A-Za-z]*)", name))
        name = ' '.join(re.split(r"([0-9][0-9]*)", name))
        # нижний регистр
        name = name.lower()
        # удаление пунктуации
        name = re.sub(r"[^а-яa-z\d\s]+", ' ', name)        
        # удаление слова "prosept"
        name = re.sub(r"prosept", ' ', name)
        # удаление стоп-слов
        name = ' '.join([word for word in name.split() if word not in stop_words])    
    else:
        name = ''    
    return name 

In [11]:
%%time
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")
#import pandas as pd
#import modin.pandas as pd
#ray.init(num_cpus=10)
from transformers import BertForPreTraining, BertTokenizerFast, BertConfig

list_func = [clean_texts]
list_columns = [['name','ozon_name','name_1c','wb_name']]

best_param = []
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")


for func in list_func:
    for columns in list_columns:
        for len_token in range(61,65,1):
        
        

            def t_fit_LaBSE(df,func,df_columns=['name']):


                df_tmp = df[df_columns[0]].apply(func)

                if len(df_columns)>1:
                    for i in range(1,len(df_columns)):
                        df_tmp = df_tmp + ' ' + df[df_columns[i]].apply(func)


                list_tmp = df_tmp.tolist()
                #print(list_tmp)
                encoded_input = tokenizer(list_tmp
                                          , padding=True
                                          , truncation=True
                                          , max_length=len_token
                                          , return_tensors='pt')
                with torch.no_grad():
                    model_output = model(**encoded_input)
                embeddings = model_output.pooler_output
                embeddings = torch.nn.functional.normalize(embeddings)            




                #model = model_LaBSE.encode(df_tmp)

                return embeddings,df[['id','name']]

            #Получим вектора из датафрейма df_product.
            product_embedding_LaBSE, df_product_LaBSE =  t_fit_LaBSE(df_product,func,columns)
            print(1)


            def t_predict_LaBSE(txt):          
                encoded_input = tokenizer(func(txt)
                                          , padding=True
                                          , truncation=True
                                          , max_length=len_token
                                          , return_tensors='pt')
                with torch.no_grad():
                    model_output = model(**encoded_input)
                embeddings = model_output.pooler_output
                txt_embedding = torch.nn.functional.normalize(embeddings)             


                return util.pytorch_cos_sim(txt_embedding,product_embedding_LaBSE)

            df_predict_LaBSE = df_valid['product_name'].apply(t_predict_LaBSE)

            # индексы лучших совпадений для строк
            top_k_matches = []
            for i in range(df_predict_LaBSE.shape[0]):
                _,indices = df_predict_LaBSE.iloc[i].topk(len(df_product))
                top_k_matches.append(indices)

            #Вычислим метрики для данной модели.
            df_predict = pd.DataFrame()
            df_predict['index'] = df_valid['index']
            df_predict['predict'] = top_k_matches

            n=5
            def t_compare_LaBSE(df):

                return (df['index'] in (df['predict'][0][0:n]))
            n=1
            accuracy_in1_LaBSE='{:.3f}'.format(df_predict[['index','predict']].apply(t_compare_LaBSE,axis=1).sum()*100/df_predict.shape[0])
            n=5
            accuracy_in5_LaBSE='{:.3f}'.format(df_predict[['index','predict']].apply(t_compare_LaBSE,axis=1).sum()*100/df_predict.shape[0])


            def t_compare_MRR_LaBSE(df):
                return (1/(list(df['predict'][0]).index(df['index'])+1))

            MRR_LaBSE = '{:.3f}'.format(df_predict[['index','predict']].apply(t_compare_MRR_LaBSE,axis=1).sum()/df_predict.shape[0])
            best_param.append([accuracy_in1_LaBSE,accuracy_in5_LaBSE, MRR_LaBSE,len_token])
            print(accuracy_in1_LaBSE,accuracy_in5_LaBSE, MRR_LaBSE,len_token)
            tmp_str = 'all' if len(columns)>1 else 'name'
            df_result_valid = pd.concat([df_result_valid, pd.DataFrame({'Имя модели':[\
                            f'Модель LaBSE c функцией очистки:{func.__name__},столбцы:{tmp_str}']\
                            ,'Дообучение':[False]\
                            ,'Accuracy_in1':[accuracy_in1_LaBSE]\
                            ,'Accuracy_in5':[accuracy_in5_LaBSE]\
                            ,'MRR':[MRR_LaBSE]})],ignore_index=True)

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.pr

1
75.088 94.700 0.839 61
1
75.442 94.523 0.840 62
1
75.972 94.346 0.842 63
1


KeyboardInterrupt: 

CPU times: total: 7min 16s
Wall time: 49.4 s

In [ ]:
75.088 94.700 0.839 61

[Содержание](#Содержание) 

In [53]:
df_dealerprice.head()

,id,product_key,price,product_url,product_name,date,dealer_id
0,2,546227,233.00,https://akson.ru//p/sredstvo_universalnoe_prosept_universal_spray_500ml/,"Средство универсальное Prosept Universal Spray, 500мл",2023-07-11,2
1,3,546408,175.00,https://akson.ru//p/kontsentrat_prosept_multipower_dlya_mytya_polov_tsitrus_1l/,"Концентрат Prosept Multipower для мытья полов, цитрус 1л",2023-07-11,2
2,4,546234,285.00,https://akson.ru//p/sredstvo_dlya_chistki_lyustr_prosept_universal_anti_dust_500ml/,"Средство для чистки люстр Prosept Universal Anti-dust, 500мл",2023-07-11,2
3,5,651258,362.00,https://akson.ru//p/udalitel_rzhavchiny_prosept_rust_remover_0_5l_023_05/,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05",2023-07-11,2
4,6,546355,205.00,https://akson.ru//p/sredstvo_moyushchee_dlya_bani_i_sauny_prosept_multipower_wood_1l/,Средство моющее для бани и сауны Prosept Multipower Wood 1л,2023-07-11,2


In [94]:
def clean_texts(name):
    '''
    Функции очистки текста.
    Принимает на вход строку - название товара,
    возвращает его в отредактированном виде
    '''
    # стоп-слова для английского и русского языков
    stop_words_en = set(stopwords.words('english'))
    stop_words_ru = set(stopwords.words('russian'))
    # объединим стоп-слова
    stop_words = stop_words_en.union(stop_words_ru)
    
    if not pd.isna(name):
        # разделение слов
        name = ' '.join(re.split(r"([A-Za-z][A-Za-z]*)", name))
        name = ' '.join(re.split(r"([А-ЯЁа-яё][А-ЯЁа-яё]*)", name)) #
        #name = ' '.join(re.split(r"([0-9][0-9]*)", name))
        # нижний регистр
        name = name.lower()
        # удаление пунктуации
        #name = re.sub(r"[^а-яa-z\d\s]+", ' ', name)        
        # удаление слова "prosept"
        name = re.sub(r"prosept", ' ', name)
        # удаление стоп-слов
        name = ' '.join([word for word in name.split() if word not in stop_words])    
    else:
        name = ''    
    return name

In [95]:
df_tmp = pd.DataFrame()
df_tmp['product_name'] = df_dealerprice['product_name'].apply(clean_texts)

In [96]:
df_tmp.head()

,product_name
0,"средство универсальное universal spray , 500 мл"
1,"концентрат multipower мытья полов , цитрус 1 л"
2,"средство чистки люстр universal anti - dust , 500 мл"
3,"удалитель ржавчины rust remover 0,5 л 023-05"
4,средство моющее бани сауны multipower wood 1 л


In [99]:
def find_weight(txt):
    res = ''
    match_dictionary = ['л','мл','г','кг']
    for i in match_dictionary:
        if (res_num:=txt.find(' '+i+' '))!=-1:
            res = i
    for i in match_dictionary:
        if (res_num:=txt.find(' '+i))!=-1:
            res = i    
    return [res,res_num]
            

In [100]:
df_tmp2['weight'] = df_tmp2['product_name'].apply(find_weight)


             weight
[л, -1]    0.536295
[г, -1]    0.177704
[мл, -1]   0.118192
[, -1]     0.049716
[кг, 45]   0.008670
...             ...
[кг, 109]  0.000196
[кг, 14]   0.000196
[кг, 108]  0.000147
[кг, 88]   0.000147
[кг, 66]   0.000049

[66 rows x 1 columns]


In [ ]:
class_frequency = df_tmp['weight'].value_counts(normalize=True).to_frame()
print(class_frequency)

In [88]:
df_tmp.head()

,product_name,weight
0,"средство универсальное universal spray , 500 мл",мл
1,"концентрат multipower мытья полов, цитрус 1 л",л
2,"средство чистки люстр universal anti - dust , 500 мл",мл
3,"удалитель ржавчины rust remover 0 , 5 л 023 - 05",л
4,средство моющее бани сауны multipower wood 1 л,л


In [90]:
df_tmp[df_tmp['weight']=='']

,product_name,weight
32,sb base,
822,концентрат crystal rinser японский чай,
937,"""нейтрализатор запаха flox sea """"морской бриз"""", 0 , 5 """,
941,усиленное средство ухода сантехникой bath acid plus концентрат,
974,средство комплексного мытья отбеливания поверхностей duty belizna дезинфицирующим эф,
...,...,...
20306,строительный антисептик,
20315,грунтовка укрепляющий 32 укрепляющая,
20335,строительный антисептик,
20381,строительный антисептик невымываемый,


[Содержание](#Содержание) 

In [38]:
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)
df_result_valid

,Имя модели,Дообучение,Accuracy_in1,Accuracy_in5,MRR
0,"Модель LaBSE c функцией очистки:str_edit_clean,столбцы:all",False,74.028,92.580,0.823
1,"Модель LaBSE c функцией очистки:str_edit_clean,столбцы:name",False,67.314,90.989,0.781
2,"Модель LaBSE c функцией очистки:str_edit2_clean,столбцы:all",False,71.201,91.873,0.805
3,"Модель LaBSE c функцией очистки:str_edit2_clean,столбцы:name",False,68.728,90.636,0.780
4,"Модель LaBSE c функцией очистки:clean_texts,столбцы:all",False,72.615,92.049,0.812
5,"Модель LaBSE c функцией очистки:clean_texts,столбцы:name",False,63.251,88.163,0.745
6,"Модель LaBSE c функцией очистки:no_change,столбцы:all",False,71.025,92.049,0.804
7,"Модель LaBSE c функцией очистки:no_change,столбцы:name",False,61.661,89.399,0.743
8,"Модель LaBSE c функцией очистки:str_edit,столбцы:all",False,73.322,93.110,0.820
9,"Модель LaBSE c функцией очистки:str_edit,столбцы:name",False,65.194,90.636,0.768


Параметры показавшие наилучший результат:  
- использование всех столбцов 'name' из датафрейма 'df_product'.  
- использование одной из функции обработки str_edit_clean или str_edit.  

[Содержание](#Содержание) 

<a name="LaBSE_CatBoost"></a>
#### 3.3.1. Модель LaBSE + CatBoost.

Обновим индексы в тренировочной, валидационной и тестовой выборках для прохождения encode LaBSE

In [111]:

print(embeddings)

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[-0.0714, -0.0772, -0.0305,  ...,  0.0136, -0.0402, -0.0245],
        [-0.0644, -0.0687, -0.0521,  ...,  0.0139, -0.0362, -0.0213]])


In [108]:
!pip install tensorflow

     -------------------------------------- 300.8/300.8 MB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 8.7 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 8.3 MB/s eta 0:00:00
     -------------------------------------- 938.4/938.4 kB 6.6 MB/s eta 0:00:00
     -------------------------------------- 442.0/442.0 kB 4.6 MB/s eta 0:00:00
     --------------------------------------- 24.4/24.4 MB 10.1 MB/s eta 0:00:00
     -------------------------------------- 65.5/65.5 kB 876.9 kB/s eta 0:00:00
     -------------------------------------- 130.2/130.2 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 10.4 MB/s eta 0:00:00
     -------------------------------------- 57.5/57.5 kB 605.5 kB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 7.3 MB/s eta 0:00:00
     ------------------------------------ 102.2/102.2 kB 843.2 kB/s eta 0:00:00
     -------------------------------------- 226

In [107]:
import tensorflow as tf
import tensorflow_text  # noqa
import tensorflow_hub as hub

# Loading models from tfhub.dev
encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang/1")
preprocessor = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang_preprocess/1")

# Constructing model to encode texts into high-dimensional vectors
sentences = tf.keras.layers.Input(shape=(), dtype=tf.string, name="sentences")
encoder_inputs = preprocessor(sentences)
sentence_representation = encoder(encoder_inputs)["pooled_output"]
normalized_sentence_representation = tf.nn.l2_normalize(sentence_representation, axis=-1)  # for cosine similarity
model = tf.keras.Model(sentences, normalized_sentence_representation)
model.summary()

ModuleNotFoundError: No module named 'tensorflow'

In [39]:
df_train_c = df_train.reset_index(drop= True)
df_valid_c = df_valid.reset_index(drop= True)
df_test_c = df_test.reset_index(drop= True)

In [103]:
%%time
# загрузка предварительно обученной модели LaBSE
model_LaBSE = SentenceTransformer('LaBSE')

columns = ['name','ozon_name','name_1c','wb_name']
columns_dealer = ['product_name']


for func in [str_edit_clean]:
    # преобразование текстов в векторы для датафреймов df_train,df_valid по столбцу 'product_name'  
    # и df_product по столбцам 'name'
    def t_fit_LaBSE(df,func,df_columns=['name']):

        df_tmp = df[df_columns[0]].apply(func)
        
        if len(df_columns)>1:
            for i in range(1,len(df_columns)):
                df_tmp = df_tmp + ' ' + df[df_columns[i]].apply(func)
        
        
        model = model_LaBSE.encode(df_tmp)
        
        return model
    
    
    product_embedding_LaBSE_Cat =  t_fit_LaBSE(df_product,func,columns)
    
    dealer_embedding_LaBSE_Cat_train =  t_fit_LaBSE(df_train_c,func,columns_dealer)
    
    dealer_embedding_LaBSE_Cat_valid =  t_fit_LaBSE(df_valid_c,func,columns_dealer)
    
    #texts_producer_embeddings = model_LaBSE.encode(texts_producer.tolist(), convert_to_tensor=True)
    #texts_dealer_embeddings = model_LaBSE.encode(texts_dealer.tolist(), convert_to_tensor=True)

    # измерение схожести (косинусное сходство)
    similarity_matrix_train = util.pytorch_cos_sim(dealer_embedding_LaBSE_Cat_train, product_embedding_LaBSE_Cat)

CPU times: total: 8min 30s
Wall time: 1min


In [41]:
matches =10
# индексы лучших совпадений для строк
top_k_matches = []
for i in range(similarity_matrix_train.shape[0]):
    _, indices = similarity_matrix_train[i].topk(matches)
    top_k_matches.append(indices)

In [42]:
# таблица с результатом
results_list = []
for i, indices in enumerate(top_k_matches):
    for rank, match_index in enumerate(indices, start=1):
        results_list.append({
            'Row': i + 1,
            'Key': df_train_c['key'].iloc[i],
            'Product_Name': df_train_c['product_name'].iloc[i],
            'Best_Match_Index': match_index,
            'Best_Match_Id': df_product.loc[match_index.item(), 'id'],
            'Best_Match_Name': df_product.loc[match_index.item(), 'name'],
            'Similarity': similarity_matrix_train[i, match_index].item(),
        })

        # добавим значения из similarity_matrix
        for col in range(similarity_matrix_train.shape[1]):
            results_list[-1][f'Similarity_{col}'] = similarity_matrix_train[i, col].item()
results_df_train = pd.DataFrame(results_list)
results_df_train

,Row,Key,Product_Name,Best_Match_Index,Best_Match_Id,Best_Match_Name,Similarity,Similarity_0,Similarity_1,Similarity_2,...,Similarity_484,Similarity_485,Similarity_486,Similarity_487,Similarity_488,Similarity_489,Similarity_490,Similarity_491,Similarity_492,Similarity_493
0,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(210),321,"Лак для бани и сауны PROSEPTготовый состав / 0,9 л",0.775264,0.420175,0.308708,0.311293,...,0.377918,0.508324,0.531196,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431
1,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(219),322,Лак для бани и сауны PROSEPTготовый состав / 2 л,0.638258,0.420175,0.308708,0.311293,...,0.377918,0.508324,0.531196,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431
2,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(100),433,"Кожный антисептик PROF DZ готовый состав / 0,1 л",0.630521,0.420175,0.308708,0.311293,...,0.377918,0.508324,0.531196,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431
3,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(311),65,"Универсальный концентрат для санитарных комнат с антимикробным эффектомBath Uni готовый состав / 0,5 л",0.629195,0.420175,0.308708,0.311293,...,0.377918,0.508324,0.531196,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431
4,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(437),330,"Лак паркетный PROSEPT, полуматовый,готовый состав / 0,9 л",0.626572,0.420175,0.308708,0.311293,...,0.377918,0.508324,0.531196,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5655,566,18382039,"Акриловый герметик для швов в деревянных домах, конструкциях, изделиях PROSEPT цвет орех, 7 кг 024-7 о",tensor(226),462,"Герметик акриловый цвет орех, 15 кг.",0.713248,0.401221,0.358470,0.665201,...,0.365270,0.352529,0.274565,0.303405,0.387062,0.320383,0.332493,0.354460,0.346438,0.331355
5656,566,18382039,"Акриловый герметик для швов в деревянных домах, конструкциях, изделиях PROSEPT цвет орех, 7 кг 024-7 о",tensor(180),227,"Герметик акриловый цвет тик, 3 кг.",0.705463,0.401221,0.358470,0.665201,...,0.365270,0.352529,0.274565,0.303405,0.387062,0.320383,0.332493,0.354460,0.346438,0.331355
5657,566,18382039,"Акриловый герметик для швов в деревянных домах, конструкциях, изделиях PROSEPT цвет орех, 7 кг 024-7 о",tensor(272),463,"Герметик акриловый цвет тик, 3 кг.",0.705463,0.401221,0.358470,0.665201,...,0.365270,0.352529,0.274565,0.303405,0.387062,0.320383,0.332493,0.354460,0.346438,0.331355
5658,566,18382039,"Акриловый герметик для швов в деревянных домах, конструкциях, изделиях PROSEPT цвет орех, 7 кг 024-7 о",tensor(223),226,"Герметик акриловый цвет Сосна, 3 кг",0.702630,0.401221,0.358470,0.665201,...,0.365270,0.352529,0.274565,0.303405,0.387062,0.320383,0.332493,0.354460,0.346438,0.331355


In [43]:
# проставим реальные значения из мэтчинга заказчика
merged_df_train = results_df_train.merge(df_train_c[['key', 'product_id']], left_on='Key', right_on='key', how='left')

# переименование столбца
merged_df_train = merged_df_train.rename(columns={'product_id': 'Real_Product_Id'})
merged_df_train

,Row,Key,Product_Name,Best_Match_Index,Best_Match_Id,Best_Match_Name,Similarity,Similarity_0,Similarity_1,Similarity_2,...,Similarity_486,Similarity_487,Similarity_488,Similarity_489,Similarity_490,Similarity_491,Similarity_492,Similarity_493,key,Real_Product_Id
0,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(210),321,"Лак для бани и сауны PROSEPTготовый состав / 0,9 л",0.775264,0.420175,0.308708,0.311293,...,0.531196,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431,26391212,321
1,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(219),322,Лак для бани и сауны PROSEPTготовый состав / 2 л,0.638258,0.420175,0.308708,0.311293,...,0.531196,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431,26391212,321
2,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(100),433,"Кожный антисептик PROF DZ готовый состав / 0,1 л",0.630521,0.420175,0.308708,0.311293,...,0.531196,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431,26391212,321
3,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(311),65,"Универсальный концентрат для санитарных комнат с антимикробным эффектомBath Uni готовый состав / 0,5 л",0.629195,0.420175,0.308708,0.311293,...,0.531196,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431,26391212,321
4,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(437),330,"Лак паркетный PROSEPT, полуматовый,готовый состав / 0,9 л",0.626572,0.420175,0.308708,0.311293,...,0.531196,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431,26391212,321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,566,18382039,"Акриловый герметик для швов в деревянных домах, конструкциях, изделиях PROSEPT цвет орех, 7 кг 024-7 о",tensor(226),462,"Герметик акриловый цвет орех, 15 кг.",0.713248,0.401221,0.358470,0.665201,...,0.274565,0.303405,0.387062,0.320383,0.332493,0.354460,0.346438,0.331355,18382039,375
5956,566,18382039,"Акриловый герметик для швов в деревянных домах, конструкциях, изделиях PROSEPT цвет орех, 7 кг 024-7 о",tensor(180),227,"Герметик акриловый цвет тик, 3 кг.",0.705463,0.401221,0.358470,0.665201,...,0.274565,0.303405,0.387062,0.320383,0.332493,0.354460,0.346438,0.331355,18382039,375
5957,566,18382039,"Акриловый герметик для швов в деревянных домах, конструкциях, изделиях PROSEPT цвет орех, 7 кг 024-7 о",tensor(272),463,"Герметик акриловый цвет тик, 3 кг.",0.705463,0.401221,0.358470,0.665201,...,0.274565,0.303405,0.387062,0.320383,0.332493,0.354460,0.346438,0.331355,18382039,375
5958,566,18382039,"Акриловый герметик для швов в деревянных домах, конструкциях, изделиях PROSEPT цвет орех, 7 кг 024-7 о",tensor(223),226,"Герметик акриловый цвет Сосна, 3 кг",0.702630,0.401221,0.358470,0.665201,...,0.274565,0.303405,0.387062,0.320383,0.332493,0.354460,0.346438,0.331355,18382039,375


In [44]:
# создадим target
merged_df_train['target'] = (merged_df_train['Best_Match_Id'] == merged_df_train['Real_Product_Id']).astype(int)
merged_df_train.head()

,Row,Key,Product_Name,Best_Match_Index,Best_Match_Id,Best_Match_Name,Similarity,Similarity_0,Similarity_1,Similarity_2,...,Similarity_487,Similarity_488,Similarity_489,Similarity_490,Similarity_491,Similarity_492,Similarity_493,key,Real_Product_Id,target
0,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(210),321,"Лак для бани и сауны PROSEPTготовый состав / 0,9 л",0.775264,0.420175,0.308708,0.311293,...,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431,26391212,321,1
1,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(219),322,Лак для бани и сауны PROSEPTготовый состав / 2 л,0.638258,0.420175,0.308708,0.311293,...,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431,26391212,321,0
2,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(100),433,"Кожный антисептик PROF DZ готовый состав / 0,1 л",0.630521,0.420175,0.308708,0.311293,...,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431,26391212,321,0
3,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(311),65,"Универсальный концентрат для санитарных комнат с антимикробным эффектомBath Uni готовый состав / 0,5 л",0.629195,0.420175,0.308708,0.311293,...,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431,26391212,321,0
4,1,26391212,Лак для бани и сауны Prosept 0.9л,tensor(437),330,"Лак паркетный PROSEPT, полуматовый,готовый состав / 0,9 л",0.626572,0.420175,0.308708,0.311293,...,0.463842,0.392611,0.362444,0.408765,0.313022,0.418854,0.471431,26391212,321,0


Выделим столбцы для тренировки CatBoost

<a name="Выводы_CatBoost"></a>
#### 3.3.2. Выводы по полученым метрикам моделей.

Модель с дообучением не показала улучшений в Метрики, по сравнению со стационарной моделью. На данном этапе в макросервисе будет использоваться стационарная модель. Для улучшения требуется 

[Содержание](#Содержание) 

<a name="Выбор_лучшей_модели"></a>
## 4. Тестирование лучшей модели.


Рассмотрим получившиеся результаты на тестовых данных.

[Содержание](#Содержание) 

<a name="Выводы"></a>
## 5. Выводы.


<a name="Описание_решения"></a>
### 5.1. Описание решения

Выбрана предобученная модель LaBSE которая на данный момент показала наилучшие результаты.

[Содержание](#Содержание) 

<a name="Направления"></a>
### 5.2. Направления для дальнейшего исследования

1. Так как наилучшие результаты модель показала при использовании всех столбцов 'name' в датафрейме df_product, в качестве обучения модели проверить использование 'product_name' сматченных названий.
2. Выделить из имени объем (вес) продукта и использовать его как признак для модели.

*Коментарии для заказчика для улучшения модели предсказания.  
-лучше проверять таблицу product, из-за ошибок в заполнении, могут возникнуть неправильные предсказания матчинг(странно, что одинаковому названию соответствует разные id, также исключить ошибки форматирования при запллнении(article,name).
-не указана дата матчинга в датасета матчинга, что приводит к удалению дубликатов в dealerprice случайным образом, хотя скорее всего требовалась привязка по дате.

[Содержание](#Содержание) 